In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import pprint
import os


### Read CSV file(s)

In [2]:
air_df = pd.read_csv("data/location_7740/aqi.csv")

In [3]:
air_df = air_df.rename(columns={"Unnamed: 0": "time"})
air_df.head()

,time,co,no2,o3,pm10,pm25,so2
0,2019-01-01T00:00:00Z,742.0,30.8,13.4,16.8,15.2,7.9
1,2019-01-01T01:00:00Z,761.0,28.4,17.3,13.5,12.0,7.7
2,2019-01-01T02:00:00Z,776.0,29.5,19.6,23.9,22.9,7.5
3,2019-01-01T03:00:00Z,774.0,30.6,22.3,24.2,20.4,7.7
4,2019-01-01T04:00:00Z,809.0,35.5,23.6,29.0,24.9,7.7


Check null values (Later on interpolation may be needed)

In [4]:
print(air_df.isnull().values.any())

True


Check negative values (Attributes only)

In [5]:
air_df_attr = air_df.iloc[:,1:-1]
print((air_df_attr.values < 0).any())

False


### Setup and data cleaning

Directory for output files

In [6]:
os.makedirs("data/processed", exist_ok=True)

Standardize time and set index

In [7]:
air_df["time"] = pd.to_datetime(air_df["time"], utc=True).dt.tz_localize(None)
air_df = air_df.dropna(subset=["time"]).set_index("time").sort_index()

# Round to nearest hour and handle duplicates
air_df.index = air_df.index.round("h")
air_df = air_df.groupby(air_df.index).mean()

Data cutoff (Due to overly insufficient data, which is not suitable for interpolation)

In [8]:
cut_date = "2021-12-07 00:00:00"
air_df = air_df[air_df.index >= cut_date]

Remove negative values (if exist), also save a cleaned dataframe and dropped dataframe (for inavild entries)

In [9]:
numeric_cols = air_df.select_dtypes(include=[np.number]).columns
invalid_mask = (air_df[numeric_cols] < 0).any(axis=1)
dropped_air_df = air_df[invalid_mask].copy()

cleaned_air_df = air_df.copy()
cleaned_air_df[numeric_cols] = cleaned_air_df[numeric_cols].mask(cleaned_air_df[numeric_cols] < 0)

print(f"Total Rows: {len(air_df)}")
print(f"Invalid Rows identified (negatives): {len(dropped_air_df)}")

Total Rows: 34992
Invalid Rows identified (negatives): 0


### Interpolation setup

Define the full hourly range for the cleaned dataframe

In [10]:
start_date = air_df.index.min()
end_date = air_df.index.max()
full_range = pd.date_range(start=start_date, end=end_date, freq="h")

In [11]:
aligned_air_df = cleaned_air_df.reindex(full_range)

Apply time interpolation to fill in for the invalid entries

In [12]:
interpolated_air_df = aligned_air_df.interpolate(method="time", axis=0, limit_direction="both")

### Saving dataframes

In [13]:
dropped_air_df.to_csv("data/processed/dropped_air.csv")
cleaned_air_df.to_csv("data/processed/cleaned_air.csv")
interpolated_air_df.to_csv("data/processed/interpolated_air.csv")

### Interpolation Evaluation

Creating artificial random gaps from longest consecutive timestamps to calculate losses for different interpolation methods

Identify the start of each sequence of consecutive timestamps (True if the start of sequence)

In [14]:
start_of_consec_seq = air_df.index.get_level_values(0).drop_duplicates().to_series().diff() != pd.Timedelta("0 days 01:00:00")
start_of_consec_seq.head(5)

time
2021-12-07 00:00:00     True
2021-12-07 01:00:00    False
2021-12-07 02:00:00    False
2021-12-07 03:00:00    False
2021-12-07 04:00:00    False
Name: time, dtype: bool

Counter/identifer for each sequence

In [15]:
consec_seq_id = start_of_consec_seq.cumsum().to_frame(name="group")
consec_seq_id

,group
time,
2021-12-07 00:00:00,1
2021-12-07 01:00:00,1
2021-12-07 02:00:00,1
2021-12-07 03:00:00,1
2021-12-07 04:00:00,1
...,...
2025-12-07 19:00:00,5
2025-12-07 20:00:00,5
2025-12-07 21:00:00,5


Group id with their group length

In [16]:
grouped = consec_seq_id.groupby("group").size()
grouped

group
1     576
2    8736
3    8736
4    8760
5    8184
dtype: int64

Select group with the longest consecutive timestamps

In [17]:
longest_sequence = consec_seq_id[consec_seq_id["group"] == grouped.idxmax()].index
longest_sequence

DatetimeIndex(['2024-01-01 00:00:00', '2024-01-01 01:00:00',
               '2024-01-01 02:00:00', '2024-01-01 03:00:00',
               '2024-01-01 04:00:00', '2024-01-01 05:00:00',
               '2024-01-01 06:00:00', '2024-01-01 07:00:00',
               '2024-01-01 08:00:00', '2024-01-01 09:00:00',
               ...
               '2024-12-30 14:00:00', '2024-12-30 15:00:00',
               '2024-12-30 16:00:00', '2024-12-30 17:00:00',
               '2024-12-30 18:00:00', '2024-12-30 19:00:00',
               '2024-12-30 20:00:00', '2024-12-30 21:00:00',
               '2024-12-30 22:00:00', '2024-12-30 23:00:00'],
              dtype='datetime64[ns]', name='time', length=8760, freq=None)

Randomly drop 5% of timestamps (rows) from the longest sequence

In [18]:
full_seq_air = air_df.loc[longest_sequence]

In [19]:
n_drop = int(len(full_seq_air.index) * 0.05)
np.random.seed(24)
drop_indices = np.random.choice(full_seq_air.index, size=n_drop, replace=False)
drop_indices.shape

(438,)

In [20]:
missing_seq_air = full_seq_air.drop(index=drop_indices)

Re-index missing timestamps for interpolation

In [21]:
missing_seq_air = missing_seq_air.reindex(full_seq_air.index)

In [22]:
linear_interpolated_air = missing_seq_air.interpolate(
    method="linear", axis=0, limit=None, limit_direction="both")
time_interpolated_air = missing_seq_air.interpolate(
    method="time", axis=0, limit=None, limit_direction="both")
spline2_interpolated_air = missing_seq_air.interpolate(
    method="spline", axis=0, limit=None, limit_direction="both", order=2)
spline3_interpolated_air = missing_seq_air.interpolate(
    method="spline", axis=0, limit=None, limit_direction="both", order=3)

Function for calculating errors for the interpolation methods

In [23]:
def calculate_errors(original, interpolated):
    orig_arr = np.array(original)
    interp_arr = np.array(interpolated)
    
    # Filter NaNs
    mask = ~np.isnan(orig_arr) & ~np.isnan(interp_arr)
    clean_orig = orig_arr[mask]
    clean_interp = interp_arr[mask]

    if len(clean_orig) == 0:
        return np.nan, np.nan, np.nan

    mae = mean_absolute_error(clean_orig, clean_interp)
    mse = mean_squared_error(clean_orig, clean_interp)
    mape = mean_absolute_percentage_error(clean_orig, clean_interp)

    rmse = float(np.sqrt(mse))
    
    return mae, rmse, mape

def calculate_errors_all_columns(original_df, interpolated_df, interpolated_indices):
    errors_dict = {}
    for column in original_df.columns:
        if column in interpolated_df.columns:
            original_values = original_df.loc[interpolated_indices, column]
            interpolated_values = interpolated_df.loc[interpolated_indices, column]
            
            mae, rmse, mape = calculate_errors(original_values, interpolated_values)
            
            errors_dict[column] = {"MAE": mae, "RMSE": rmse, "MAPE": mape}
    return errors_dict

interpolation_results = {
    "Linear": linear_interpolated_air,
    "Time": time_interpolated_air,
    "Spline (Ord 2)": spline2_interpolated_air,
    "Spline (Ord 3)": spline3_interpolated_air
}

Loss between interpolated and real values

In [24]:
final_scores = {}

for method_name, df_result in interpolation_results.items():
    
    method_errors = calculate_errors_all_columns(
        full_seq_air,  
        df_result,       
        drop_indices     
    )
    
    final_scores[method_name] = method_errors

for method, scores in final_scores.items():
    print(f"\n--- Results for Method: {method} ---")
    pprint.pprint(scores)


--- Results for Method: Linear ---
{'co': {'MAE': 24.867346938775512,
        'MAPE': 0.048696939853768315,
        'RMSE': 35.42351218190688},
 'no2': {'MAE': 4.974922495039682,
         'MAPE': 0.1393686227554876,
         'RMSE': 7.018269980681622},
 'o3': {'MAE': 5.09204390847248,
        'MAPE': 0.1612982260536116,
        'RMSE': 7.64792562529003},
 'pm10': {'MAE': 1.964580586080586,
          'MAPE': 0.17877812331026768,
          'RMSE': 3.6093577918036925},
 'pm25': {'MAE': 1.4242014795474327,
          'MAPE': 0.22971045945292878,
          'RMSE': 1.953588339401718},
 'so2': {'MAE': 0.37463943802063904,
         'MAPE': 0.15921338939965865,
         'RMSE': 0.6768475526843001}}

--- Results for Method: Time ---
{'co': {'MAE': 24.867346938775512,
        'MAPE': 0.048696939853768315,
        'RMSE': 35.42351218190688},
 'no2': {'MAE': 4.974922495039682,
         'MAPE': 0.13936862275548756,
         'RMSE': 7.018269980681622},
 'o3': {'MAE': 5.09204390847248,
        'MAPE':